In [1]:
import os
import math
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data
from torchtext.data import Field, BucketIterator, TabularDataset
from torch.utils.data import TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 加载数据集合

In [2]:
input_csv = './dataset/task8_train_input.csv'
output_csv = './dataset/task8_train_output.csv'
inputs = pd.read_csv(input_csv, header=None)
outputs = pd.read_csv(output_csv, header=None)

In [3]:
inputs.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,3,9,4,1,9,4,6,8,9,8,3,6,1,1,2,1,4,0,0,0
1,1,9,9,4,6,5,4,5,0,0,0,0,0,0,0,0,0,0,0,0
2,5,2,7,7,6,9,4,1,6,6,7,7,4,2,3,0,0,0,0,0
3,8,9,4,6,1,9,3,9,3,2,3,8,9,7,5,0,0,0,0,0
4,1,4,2,9,3,9,5,8,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
test_input_csv = './dataset/task8_test_input.csv'
test_inputs = pd.read_csv(test_input_csv, header=None)

In [5]:
test_inputs.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,7,3,6,7,2,4,9,8,9,8,1,0,0,0,0,0,0,0,0,0
1,9,6,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4,8,3,2,9,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0
3,9,5,2,7,7,1,8,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3,7,2,6,3,1,1,3,4,6,2,4,8,4,4,3,9,0,0,0


### 训练集与验证集划分

In [6]:
train_inputs, valid_inputs, train_outputs, valid_outputs = train_test_split(inputs, outputs, test_size=0.3, random_state=42)

In [7]:
f'train_inputs_num: {len( train_inputs)}, valid_inputs_num: {len(valid_inputs)}'

'train_inputs_num: 22400, valid_inputs_num: 9600'

#### 构建 digits 和 index 映射

In [8]:
class Digits:
    def __init__(self):
        self.digit2index = {"<PAD>": 0, "<GO>": 1, "<EOS>": 2, "<UNK>": 3}
        self.digit2count = {}
        self.index2digit = {0: "<PAD>", 1: "<GO>", 2:"<EOS>", 3:"<UNK>"}
        self.n_digits = 4
        
    def __len__(self):
        return self.n_digits

    def add_digits_seq(self, digits_sequence):
        for digit in digits_sequence:
            self.add_digit(digit)

    def add_digit(self, digit):
        if digit not in self.digit2index:
            self.digit2index[digit] = self.n_digits
            self.digit2count[digit] = 1
            self.index2digit[self.n_digits] = digit
            self.n_digits += 1
        else:
            self.digit2count[digit] += 1

In [9]:
digit_mapping = Digits()
digit_mapping.add_digits_seq('123456789')
digit2index = digit_mapping.digit2index
index2digit = digit_mapping.index2digit

In [10]:
index2digit

{0: '<PAD>',
 1: '<GO>',
 2: '<EOS>',
 3: '<UNK>',
 4: '1',
 5: '2',
 6: '3',
 7: '4',
 8: '5',
 9: '6',
 10: '7',
 11: '8',
 12: '9'}

#### 利用 DigitSeqDataset 包装数据并生成 batch 

In [11]:
class DigitSeqDataset(data.Dataset):
    """DigitSeqDataset"""

    def __init__(self, inputs, outputs):
        self.input_digit_seq_dataframe = inputs
        self.output_digit_seq_dataframe = outputs

        
    def pad_sentence(self, digits_sequence, max_digits_len, pad_token_id):
        '''对 batch 中的 digits序列进行补全，保证 batch 中的每行都有相同的 sequence_length
        
        参数：
        - sentence batch
        - pad_token_id: <PAD> 对应索引号
        '''
        # max_digits = max([len(digits) for digits in digits_seq_batch])
        return [digits_sequence + [pad_token_id] * (max_digits_len - len(digits_sequence))]
    
    def __len__(self):
        return len(self.input_digit_seq_dataframe)
    

    def __getitem__(self, idx):
        """
        """
        input_digit_seq = self.input_digit_seq_dataframe.iloc[idx, :].as_matrix()
    
        input_digit2index_seq = [digit2index.get(str(i), digit2index.get('UNK')) for i in input_digit_seq if i != 0]
        
        input_digit2index_seq.insert(0, digit2index.get('<GO>'))
        
        input_digit2index_seq.append(digit2index.get('<EOS>'))
        
        input_digit2index_seq = self.pad_sentence(input_digit2index_seq, 22, digit2index.get('<PAD>'))
        
        input_digit2index_seq_tensor = torch.tensor(input_digit2index_seq)#.view(1, -1)
        
        
        
        output_digit_seq = self.output_digit_seq_dataframe.iloc[idx, :].as_matrix()
        
        output_digit2index_seq = [digit2index.get(str(i), digit2index.get('<UNK>')) for i in output_digit_seq if i != 0]
        
        output_digit2index_seq.insert(0, digit2index.get('<GO>'))
        
        output_digit2index_seq.append(digit2index.get('<EOS>'))
        
        output_digit2index_seq = self.pad_sentence(output_digit2index_seq, 22, digit2index.get('<PAD>'))
        
        output_digit2index_seq_tensor = torch.tensor(output_digit2index_seq)#.view(1, -1)
        
        
        # seq_pairs = {'input_seq': input_digit_seq, 'output_seq': output_digit_seq}
        
        seq_pairs = [input_digit2index_seq, output_digit2index_seq]
        
        seq_tensor_pairs = {'input_seq_tensor': input_digit2index_seq_tensor, 'output_seq_tensor': output_digit2index_seq_tensor}
        
        return seq_tensor_pairs

In [12]:
train_input_output_dataset = DigitSeqDataset(train_inputs, train_outputs)

In [13]:
valid_input_output_dataset = DigitSeqDataset(valid_inputs, valid_outputs)

In [14]:
train_loader = torch.utils.data.DataLoader(train_input_output_dataset, batch_size=32, shuffle=False)

In [15]:
valid_loader = torch.utils.data.DataLoader(valid_input_output_dataset, batch_size=32, shuffle=False)

### 构建模型

In [16]:
class Encoder(nn.Module):
    
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.input_dim = input_dim
        
        self.emb_dim = emb_dim
        
        self.enc_hid_dim = enc_hid_dim
        
        self.dec_hid_dim = dec_hid_dim
        
        self.dropout = dropout
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional=True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        embedded = self.dropout(self.embedding(src))
        
        outputs, hidden = self.rnn(embedded)
        
        # print(outputs)
                
        #outputs = [sent len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        # initial decoder hidden is final hidden state of the forwards and backwards encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)))
        
        #outputs = [sent len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

In [17]:
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        
        self.v = nn.Parameter(torch.rand(dec_hid_dim))
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src sent len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat encoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src sent len, dec hid dim]
        #encoder_outputs = [batch size, src sent len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2))) 
        
        #energy = [batch size, src sent len, dec hid dim]
        
        energy = energy.permute(0, 2, 1)
        
        #energy = [batch size, dec hid dim, src sent len]
        
        #v = [dec hid dim]
        
        v = self.v.repeat(batch_size, 1).unsqueeze(1)
        
        #v = [batch size, 1, dec hid dim]
                
        attention = torch.bmm(v, energy).squeeze(1)
        
        #energy = [batch size, src len]
        
        return F.softmax(attention, dim=1)

In [18]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src sent len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src sent len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim=2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [sent len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #sent len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        output = self.out(torch.cat((output, weighted, embedded), dim=1))
        
        #output = [bsz, output dim]
        
        return output, hidden.squeeze(0)

In [19]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        
        #src = [src sent len, batch size]
        #trg = [trg sent len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        
        # max_len = trg.shape[0]
        max_len = src.shape[0]
    
        vocab_size = self.decoder.output_dim
        
        # tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, vocab_size).to(self.device)
        
        # encoder_outputs is all hidden states of the input sequence, back and forwards
        # hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        # first input to the decoder is the <GO> tokens
        output = torch.tensor([digit2index.get('<GO>')] * batch_size) 
        
    
        for t in range(1, max_len):
            
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            
            outputs[t] = output
            
            teacher_force = random.random() < teacher_forcing_ratio
            
            top1 = output.max(1)[1]
            
            output = (trg[t] if teacher_force else top1)

        return outputs

### Training the Seq2Seq Model

In [20]:
INPUT_DIM = len(digit2index)
OUTPUT_DIM = len(digit2index)

ENC_EMB_DIM = 8
DEC_EMB_DIM = 8

ENC_HID_DIM = 32
DEC_HID_DIM = 32

ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [21]:
optimizer = optim.Adam(model.parameters())

In [22]:
pad_idx = 0

criterion = nn.CrossEntropyLoss(ignore_index=0)

In [23]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.get('input_seq_tensor').squeeze().permute(1,0)#.view(22, 32)#.view(32,-1,1)
        trg = batch.get('output_seq_tensor').squeeze().permute(1,0)#.view(22, 32)#.view(32,-1,1)
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        loss = criterion(output[1:].view(-1, output.shape[2]), trg[1:].contiguous().view(-1))
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [24]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):
            
            src = batch.get('input_seq_tensor').squeeze().permute(1, 0) #.view(22, 32)#.view(32,-1,1)
            trg = batch.get('output_seq_tensor').squeeze().permute(1, 0) #.view(22, 32)#.view(32,-1,1)
            

            #turn off teacher forcing
            output = model(src, trg, 0) 

            loss = criterion(output[1:].view(-1, output.shape[2]), trg[1:].contiguous().view(-1))

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [25]:
index2digit[0]

'<PAD>'

In [26]:
EPOCHS = 3

CLIP = 10

SAVE_DIR = './checkpoints/'

MODEL_SAVE_PATH = os.path.join(SAVE_DIR, 'seq2seq_attention_model.pt')

best_valid_loss = float('inf')

if not os.path.isdir(f'{SAVE_DIR}'):
    os.makedirs(f'{SAVE_DIR}')
    
train_iterator = train_loader
valid_iterator = valid_loader


for epoch in range(EPOCHS):
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), MODEL_SAVE_PATH)
    
    print(f'| Epoch: {epoch+1:03} | Train Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f} | Val. Loss: {valid_loss:.3f} | Val. PPL: {math.exp(valid_loss):7.3f} |')

| Epoch: 001 | Train Loss: 1.875 | Train PPL:   6.522 | Val. Loss: 0.897 | Val. PPL:   2.452 |
| Epoch: 002 | Train Loss: 0.631 | Train PPL:   1.879 | Val. Loss: 0.179 | Val. PPL:   1.196 |
| Epoch: 003 | Train Loss: 0.326 | Train PPL:   1.385 | Val. Loss: 0.081 | Val. PPL:   1.085 |


### 预测 test 数据集

In [27]:
model.load_state_dict(torch.load(MODEL_SAVE_PATH))

In [28]:
model

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(13, 8)
    (rnn): GRU(8, 32, bidirectional=True)
    (fc): Linear(in_features=64, out_features=32, bias=True)
    (dropout): Dropout(p=0.5)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=96, out_features=32, bias=True)
    )
    (embedding): Embedding(13, 8)
    (rnn): GRU(72, 32)
    (out): Linear(in_features=104, out_features=13, bias=True)
    (dropout): Dropout(p=0.5)
  )
)

In [29]:
class DigitSeqTestDataset(data.Dataset):

    def __init__(self, inputs):
        self.input_digit_seq_dataframe = inputs

        
    def pad_sentence(self, digits_sequence, max_digits_len, pad_token_id):
        return [digits_sequence + [pad_token_id] * (max_digits_len - len(digits_sequence))]
    
    def __len__(self):
        return len(self.input_digit_seq_dataframe)
    

    def __getitem__(self, idx):
        
        input_digit_seq = self.input_digit_seq_dataframe.iloc[idx, :].as_matrix()
    
        input_digit2index_seq = [digit2index.get(str(i), digit2index.get('UNK')) for i in input_digit_seq if i != 0]
        
        input_digit2index_seq.insert(0, digit2index.get('<GO>'))
        
        input_digit2index_seq.append(digit2index.get('<EOS>'))
        
        input_digit2index_seq = self.pad_sentence(input_digit2index_seq, 22, digit2index.get('<PAD>'))
        
        input_digit2index_seq_tensor = torch.tensor(input_digit2index_seq)
        
        
        seq_tensor_pairs = {'input_seq_tensor': input_digit2index_seq_tensor}
        
        return seq_tensor_pairs

#### 加载 task8_test_input.csv

In [30]:
test_input_csv = './dataset/task8_test_input.csv'
test_inputs = pd.read_csv(test_input_csv, header=None)

In [31]:
test_input_dataset = DigitSeqTestDataset(test_inputs)

In [32]:
test_loader = torch.utils.data.DataLoader(test_input_dataset, batch_size=32, shuffle=False)

In [33]:
len(test_loader)

1000

In [34]:
test_iterator = test_loader

In [35]:
def predict(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    preds = []
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):
            
            src = batch.get('input_seq_tensor').squeeze().permute(1, 0)
            # trg = batch.get('output_seq_tensor').squeeze().permute(1, 0)

            trg = None
            

            #turn off teacher forcing
            output = model(src, trg, 0) 
            
            
            topv, topi = output.data.topk(1)
        

            decoder_output = topi.squeeze().detach()
            decoder_output = decoder_output[1:].permute(1, 0)

            preds.append([digit_idx.item() for seq in decoder_output for digit_idx in seq])
        
    return preds

In [36]:
results = predict(model, test_iterator,criterion)

In [37]:
len(results)

1000

In [38]:
pred_array = np.array([index2digit[item] for batch in results for item in batch]).reshape(1000,32,21).reshape(32000,21)

### 保存预测结果

In [39]:
pd.DataFrame([list("".join(list(i)).replace('<EOS>','0')[:-1]) for i in pred_array]).to_csv('reverse_seq_predictions .csv', header=None,index=False)